In [9]:
#Install Packages
!pip install nfl_data_py

In [10]:
# *** IMPORT LIBRARIES ****
import nfl_data_py as nfl
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [11]:
# *** FUNCTION TO READ IN NFL PLAY-BY-PLAY DATA FROM REMOTE SERVER ***
# ONLY NEEDS TO BE USED ONCE
def remote_data():
  years2016_2022 = [2016, 2017, 2018, 2019, 2020, 2021, 2022]
  df2016_2022 = nfl.import_pbp_data(years2016_2022)
  df2023 = nfl.import_pbp_data([2023])
  return (df2016_2022, df2023)

In [12]:
# *** FUNCTION TO FILTER DATAFRAMES,
# INPUT: TRAINING AND TESTING DATA
# OUTPUT: FILTERED X AND Y TRAINING AND TESTING DATA ***
# ONLY NEEDS TO BE USED ONCE

def filter_data(df2016_2022, df2023):
  filtered_df = df2016_2022[['down','half_seconds_remaining','yardline_100','ydstogo','goal_to_go',
                        'ep','wp','score_differential','game_half','posteam_timeouts_remaining',
                        'defteam_timeouts_remaining','game_seconds_remaining','home_score','away_score','home_team',
                        'away_team','posteam','defteam','home_coach','away_coach','season']]
  filtered_df = filtered_df.dropna()

  filtered_df['UNDER_TWO_MINUTES'] = 0  # Initialize 'UnderTwoMinutes' column with zeros
  filtered_df.loc[filtered_df['half_seconds_remaining'] <= 120, 'UNDER_TWO_MINUTES'] = 1

  filtered_df['HOME_TEAM_WINS'] = 0
  filtered_df.loc[filtered_df['home_score'] > filtered_df['away_score'], 'HOME_TEAM_WINS'] = 1

  filtered_df['AWAY_TEAM_WINS'] = 0
  filtered_df.loc[filtered_df['home_score'] < filtered_df['away_score'], 'AWAY_TEAM_WINS'] = 1

  filtered_df['POSTEAM_WINS'] = 0
  filtered_df.loc[(filtered_df['home_team'] == filtered_df['posteam']) & (filtered_df['HOME_TEAM_WINS'] == 1), 'POSTEAM_WINS'] = 1
  filtered_df.loc[(filtered_df['away_team'] == filtered_df['posteam']) & (filtered_df['AWAY_TEAM_WINS'] == 1), 'POSTEAM_WINS'] = 1

  filtered_df['H2'] = 0
  filtered_df.loc[filtered_df['game_half'] == 'Half2', 'H2'] = 1

  filtered_df['OT'] = 0
  filtered_df.loc[filtered_df['game_half'] == 'Overtime', 'OT'] = 1

  X2016_2022 = filtered_df[['down','half_seconds_remaining','yardline_100','goal_to_go','ydstogo',
                 'UNDER_TWO_MINUTES','score_differential','game_seconds_remaining','H2','OT',
                'posteam_timeouts_remaining','defteam_timeouts_remaining']]

  y2016_2022 = filtered_df['POSTEAM_WINS']



  filtered_df2 = df2023[['down','half_seconds_remaining','yardline_100','ydstogo','goal_to_go',
                        'ep','wp','score_differential','game_half','posteam_timeouts_remaining',
                        'defteam_timeouts_remaining','game_seconds_remaining','home_score','away_score','home_team',
                        'away_team','posteam','defteam','home_coach','away_coach','season']]
  filtered_df2 = filtered_df2.dropna()

  filtered_df2['UNDER_TWO_MINUTES'] = 0  # Initialize 'UnderTwoMinutes' column with zeros
  filtered_df2.loc[filtered_df2['half_seconds_remaining'] <= 120, 'UNDER_TWO_MINUTES'] = 1

  filtered_df2['HOME_TEAM_WINS'] = 0
  filtered_df2.loc[filtered_df2['home_score'] > filtered_df2['away_score'], 'HOME_TEAM_WINS'] = 1

  filtered_df2['AWAY_TEAM_WINS'] = 0
  filtered_df2.loc[filtered_df2['home_score'] < filtered_df2['away_score'], 'AWAY_TEAM_WINS'] = 1

  filtered_df2['POSTEAM_WINS'] = 0
  filtered_df2.loc[(filtered_df2['home_team'] == filtered_df2['posteam']) & (filtered_df2['HOME_TEAM_WINS'] == 1), 'POSTEAM_WINS'] = 1
  filtered_df2.loc[(filtered_df2['away_team'] == filtered_df2['posteam']) & (filtered_df2['AWAY_TEAM_WINS'] == 1), 'POSTEAM_WINS'] = 1

  filtered_df2['H2'] = 0
  filtered_df2.loc[filtered_df2['game_half'] == 'Half2', 'H2'] = 1

  filtered_df2['OT'] = 0
  filtered_df2.loc[filtered_df2['game_half'] == 'Overtime', 'OT'] = 1

  X2023 = filtered_df2[['down','half_seconds_remaining','yardline_100','goal_to_go','ydstogo',
                 'UNDER_TWO_MINUTES','score_differential','game_seconds_remaining','H2','OT',
                'posteam_timeouts_remaining','defteam_timeouts_remaining']]

  y2023 = filtered_df2['POSTEAM_WINS']

  return (X2016_2022, y2016_2022, X2023, y2023)

In [13]:
# *** FUNCTION TO TRAIN XGBOOST MODEL
# ONLY NEEDS TO BE USED ONCE

def XGBoost_Train(X_train, y_train):
  #scaler = StandardScaler()
  #X_train = scaler.fit_transform(X_train)

  xgb_model = xgb.XGBClassifier(n_estimators=25)
  xgb_model.fit(X_train, y_train)
  return xgb_model

In [14]:
# !!!!!!!!!!!!!!!!!!!!!! WIN PROBABILITY FUNCTION !!!!!!!!!!!!!!!!!!!!!!!!!!!!

# *** FUNCTION TO CALCULATE WIN PROBABILITIES (USING XGBOOST MODEL)
# INPUT: FITTED WIN PROBABILITY MODEL (XGBOOST); PANDAS DATAFRAME OF PLAYS WITH GAME STATE DATA
# OUTPUT: NUMPY ARRAY WITH WIN PROBABILITIES ***



def XGBoost_Predict(xgb_model, X_new):
  # Predictions
  y_pred_proba = xgb_model.predict_proba(X_new)
  y_predictions = y_pred_proba[:, 1].reshape(-1, 1)

  return y_predictions

In [15]:
df2016_2022, df2023 = remote_data()
(X2016_2022, y2016_2022, X2023, y2023) = filter_data(df2016_2022, df2023)
xgb_model = XGBoost_Train(X2016_2022, y2016_2022)

2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
Downcasting floats.
2023 done.
Downcasting floats.


In [16]:
# USE THIS CODE TO CALL WIN PROBABILITY FUNCTION
# INPUT: FITTED WIN PROBABILITY MODEL ('xgb_model'); PANDAS DATAFRAME OF PLAYS WITH GAME STATE DATA
# CAUTION: DATAFRAME COLUMNS MUST MATCH COLUMNS IN X2023
# OUTPUT: NUMPY ARRAY WITH WIN PROBABILITIES ***


predict = XGBoost_Predict(xgb_model, X2023)

In [17]:
print(X2023.columns)

Index(['down', 'half_seconds_remaining', 'yardline_100', 'goal_to_go',
       'ydstogo', 'UNDER_TWO_MINUTES', 'score_differential',
       'game_seconds_remaining', 'H2', 'OT', 'posteam_timeouts_remaining',
       'defteam_timeouts_remaining'],
      dtype='object')


In [18]:
# CALCULATED AND APPENDED STARTING WIN PROBABILITY FOR EVERY ROW

train_WP_appended = X2016_2022.copy()
test_WP_appended = X2023.copy()
train_WP_appended['CALCULATED_WP'] = XGBoost_Predict(xgb_model, X2016_2022)
test_WP_appended['CALCULATED_WP'] = XGBoost_Predict(xgb_model, X2023)

In [19]:
# POTENTIAL INPUT FILE FOR ANALYSIS
# 2023 FILTERED PLAY-BY-PLAY DATA. 4TH DOWN ONLY. PLAY TYPE = FIELD GOAL, PASS, RUN, PUNT
# INCLUDES APPENDED WIN PROBABILITY

INPUT_FILE_2023 = test_WP_appended.copy()
inner_join_df = pd.merge(df2023, INPUT_FILE_2023, left_index=True, right_index=True, how='inner')
input_play_types = ['pass','run','punt','field_goal']
INPUT_FILE_2023['play_type'] = inner_join_df['play_type']
INPUT_FILE_2023 = INPUT_FILE_2023.loc[(INPUT_FILE_2023['down'] == 4) & (INPUT_FILE_2023['play_type'].isin(input_play_types))]
INPUT_FILE_2023['CLUSTER_ASSIGNMENT'] = 0 #SHOULD WE APPEND THIS INFORMATION HERE?

<ipython-input-19-dbd20ed0458d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  INPUT_FILE_2023['CLUSTER_ASSIGNMENT'] = 0 #SHOULD WE APPEND THIS INFORMATION HERE?


In [23]:
INPUT_FILE_2023.head()

,down,half_seconds_remaining,yardline_100,goal_to_go,ydstogo,UNDER_TWO_MINUTES,score_differential,game_seconds_remaining,H2,OT,posteam_timeouts_remaining,defteam_timeouts_remaining,CALCULATED_WP,play_type,CLUSTER_ASSIGNMENT
10,4.0,1568.0,49.0,0,7.0,0,0.0,3368.0,0,0,3.0,3.0,0.454751,punt,0
17,4.0,1365.0,70.0,0,8.0,0,0.0,3165.0,0,0,3.0,3.0,0.437882,punt,0
36,4.0,956.0,11.0,0,9.0,0,-7.0,2756.0,0,0,2.0,3.0,0.302594,field_goal,0
46,4.0,811.0,36.0,0,9.0,0,-4.0,2611.0,0,0,2.0,3.0,0.301727,field_goal,0
60,4.0,384.0,85.0,0,11.0,0,-1.0,2184.0,0,0,2.0,3.0,0.368132,punt,0


In [25]:
INPUT_FILE_2023.columns

Index(['down', 'half_seconds_remaining', 'yardline_100', 'goal_to_go',
       'ydstogo', 'UNDER_TWO_MINUTES', 'score_differential',
       'game_seconds_remaining', 'H2', 'OT', 'posteam_timeouts_remaining',
       'defteam_timeouts_remaining', 'CALCULATED_WP', 'play_type',
       'CLUSTER_ASSIGNMENT'],
      dtype='object')

In [22]:
# ********* FOR DISCUSSION *********
# POTENTIAL OUTPUT FILE FROM SIMULATION MODELING FOR FINAL PROJECT ANALYSIS
# OUTPUT FILES WITH COLUMN 1 ALLOWS COMPARISON OF PERCENTAGE OF TIME TEAMS MADE EACH DECISION VERSUS OPTIMAL DECISION. CAN BE LEAGUE-WIDE, BY TEAM, BY COACH, BY OC, BY DC
# OUTPUT FILES WITH COLUMN 2 ALLOWS COMPARISON OF ACTUAL VERSUS PREDICTED PERFORMANCE
columns1 = ['posteam', 'defteam', 'WP_BEFORE', 'WP_GO_FOR_IT', 'WP_PUNT', 'WP_FIELD_GOAL', 'OPTIMAL_CHOICE', 'ACTUAL_CHOICE']
columns2 = ['WP_ACTUAL_CHOICE']

# Create empty DataFrame
output_df = pd.DataFrame(columns=columns1)
output_df

,posteam,defteam,WP_BEFORE,WP_GO_FOR_IT,WP_PUNT,WP_FIELD_GOAL,OPTIMAL_CHOICE,ACTUAL_CHOICE


In [26]:
#Clustering to determine win probabilities for various choices